# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-27.96,100,96,1.11,RU,1699653124
1,1,iqaluit,63.7506,-68.5145,-8.15,62,100,3.09,CA,1699653105
2,2,bethel,41.3712,-73.4140,9.97,61,100,0.45,US,1699653627
3,3,gadzhiyevo,69.2551,33.3362,-8.33,94,30,4.06,RU,1699653155
4,4,rio hato,8.3833,-80.1667,30.75,69,40,4.12,PA,1699653627


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = "Country"
)

# Display the map
city_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.copy() 
narrow_city_df = narrow_city_df[narrow_city_df['Max Temp'] >= 21] 
narrow_city_df = narrow_city_df[narrow_city_df['Max Temp'] <= 27] 
narrow_city_df = narrow_city_df[narrow_city_df['Wind Speed'] <= 4.5] 
narrow_city_df = narrow_city_df[narrow_city_df['Cloudiness'] == 0] 

# Drop any rows with null values
narrow_city_df = narrow_city_df.dropna() 

# Display sample data
narrow_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
128,128,taoudenni,22.6783,-3.9836,25.53,14,0,4.09,ML,1699653125
223,223,luena,-11.7833,19.9167,21.10,78,0,2.06,AO,1699653717
246,246,sirte,31.2089,16.5887,22.51,51,0,4.18,LY,1699653723
263,263,pacific grove,36.6177,-121.9166,22.99,54,0,1.34,US,1699652968
308,308,laguna,38.4210,-121.4238,21.27,39,0,4.12,US,1699653254


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
narrow_city_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df
hotel_df = hotel_df.drop(columns=['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
128,taoudenni,22.6783,-3.9836,14,ML,
223,luena,-11.7833,19.9167,78,AO,
246,sirte,31.2089,16.5887,51,LY,
263,pacific grove,36.6177,-121.9166,54,US,
308,laguna,38.4210,-121.4238,39,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories" : "accomidation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon}, {lat},{radius}"
    params["bias"] = f"proximity: {lon}, {lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


taoudenni - nearest hotel: No hotel found
luena - nearest hotel: No hotel found
sirte - nearest hotel: No hotel found
pacific grove - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
beitbridge - nearest hotel: No hotel found
illizi - nearest hotel: No hotel found
belo sur tsiribihina - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Country,Hotel Name
128,taoudenni,22.6783,-3.9836,14,ML,No hotel found
223,luena,-11.7833,19.9167,78,AO,No hotel found
246,sirte,31.2089,16.5887,51,LY,No hotel found
263,pacific grove,36.6177,-121.9166,54,US,No hotel found
308,laguna,38.4210,-121.4238,39,US,No hotel found
429,beitbridge,-22.2167,30.0000,63,ZW,No hotel found
435,illizi,26.4833,8.4667,15,DZ,No hotel found
537,belo sur tsiribihina,-19.7000,44.5500,76,MG,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)